1.Import Libraries

In [14]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler

# Hide warnings for cleaner output
pd.set_option('mode.chained_assignment', None)


2.Load RAW Dataset

In [11]:
import os

# Check if the file exists
print(os.path.exists('data/raw/retail_store_sales.csv'))


True


In [10]:
import pandas as pd

raw_data_path = r'data/raw/retail_store_sales.csv' 
df = pd.read_csv(raw_data_path)
print("First 5 rows:")
display(df.head())


First 5 rows:


,Transaction ID,Customer ID,Category,Item,Price Per Unit,Quantity,Total Spent,Payment Method,Location,Transaction Date,Discount Applied
0,TXN_6867343,CUST_09,Patisserie,Item_10_PAT,18.5,10.0,185.0,Digital Wallet,Online,2024-04-08,True
1,TXN_3731986,CUST_22,Milk Products,Item_17_MILK,29.0,9.0,261.0,Digital Wallet,Online,2023-07-23,True
2,TXN_9303719,CUST_02,Butchers,Item_12_BUT,21.5,2.0,43.0,Credit Card,Online,2022-10-05,False
3,TXN_9458126,CUST_06,Beverages,Item_16_BEV,27.5,9.0,247.5,Credit Card,Online,2022-05-07,NaN
4,TXN_4575373,CUST_05,Food,Item_6_FOOD,12.5,7.0,87.5,Digital Wallet,Online,2022-10-02,False


3.Examine Dataset

In [12]:
print("Dataset shape:", df.shape)
display(df.info())
display(df.describe())

print("Missing values per column:")
print(df.isnull().sum())

print("Number of duplicate rows:", df.duplicated().sum())

Dataset shape: (12575, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB


None

,Price Per Unit,Quantity,Total Spent
count,11966.000000,11971.000000,11971.000000
mean,23.365912,5.536380,129.652577
std,10.743519,2.857883,94.750697
min,5.000000,1.000000,5.000000
25%,14.000000,3.000000,51.000000
50%,23.000000,6.000000,108.500000
75%,33.500000,8.000000,192.000000
max,41.000000,10.000000,410.000000


Missing values per column:
Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit       609
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64
Number of duplicate rows: 0


4.Handle Missing Values

In [15]:
# Drop rows with missing 'Item'
df = df.dropna(subset=['Item'])

# Fill numeric missing values with median
num_cols = ['Price Per Unit', 'Quantity', 'Total Spent']
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())

# Fill categorical missing values with mode
df['Discount Applied'] = df['Discount Applied'].fillna(df['Discount Applied'].mode()[0])

# Remove duplicate rows
df = df.drop_duplicates()

# Save intermediate cleaned dataset
processed_dir = '/content/data/processed/'
os.makedirs(processed_dir, exist_ok=True)
df.to_csv(os.path.join(processed_dir, 'cleaned1_clustering.csv'), index=False)

print("Missing values after cleaning:", df.isnull().sum().sum())
print("Number of duplicate rows after cleaning:", df.duplicated().sum())

Missing values after cleaning: 0
Number of duplicate rows after cleaning: 0
